# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import dateparser
from datetime import datetime, timedelta
import json

## Задание 1
Будем парсить страницу со свежеми новостям на habr.com/ru/all/

In [20]:
KEYWORDS = ['python', 'парсинг']

In [21]:
habr = requests.get('https://habr.com/ru/all')
habr

<Response [200]>

In [23]:
soup = BeautifulSoup(habr.text, 'html.parser')

In [22]:
posts = soup.find_all('article', class_='post post_preview')

In [23]:
# достаем даты для каждого поста
posts_date = list(map(lambda x: x.find('span', class_='post__time').text, posts))
for i, date in enumerate(posts_date):
    if 'сегодня' in date:
        posts_date[i] = datetime.today()
    elif 'вчера' in date:
        posts_date[i] = datetime.today() - timedelta(days = 1)
    else:
        posts_date[i] = dateparser.parse(date)
posts_date

[datetime.datetime(2021, 3, 24, 14, 13, 34, 292879),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292886),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292887),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292889),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292890),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292891),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292892),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292894),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292895),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292896),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292898),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292899),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292900),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292901),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292903),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292904),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292905),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 292907),
 datetime.datetime(2021, 3, 24, 14, 13, 34, 29

In [24]:
# достаем заголовок для каждого поста
posts_title = list(map(lambda x: x.find('a', class_='post__title_link').text, posts))
posts_title

['Электроника по талонам: дефицит видеочипов, процессоров, ОЗУ, а теперь и принтеров',
 'Linux для macOS M1: что сделала команда Asahi Linux за январь-февраль 2021 (часть 2)',
 'Southbridge запускает стажировку в сфере DevOps для начинающих инженеров',
 'Windows Kernel Drivers — Стандартные ошибки – IRQL',
 'Монтажный шкаф для ЦОД. Критерии выбора. Часть 1: как определиться с размером?',
 'От int main() до BeginPlay: как происходит инициализация Unreal Engine под капотом',
 'Как устроено компьютерное зрение?',
 'DARPA: искусственный интеллект в воздушном бою истребителей F-16',
 'Оптимизация графики в Voximplant Kit',
 'Основы работы с Helm чартами и темплейтами — Часть 2',
 'Приватность в сети\xa0Биткоин: Лучшие практики',
 'Lego анонсировала выпуск самого большого и детализированного набора',
 'Одобрено NASA: образцы грунта доставит миссия с Титана',
 'Хабр ищет новостников, авторов и кураторов',
 'Сбор метрик Spring Boot приложения c помощью Prometheus и Grafana',
 'Особенности взаи

In [25]:
# достаем ссылку для каждого поста
posts_link = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), posts))
posts_link

['https://habr.com/ru/company/selectel/blog/548748/',
 'https://habr.com/ru/post/548744/',
 'https://habr.com/ru/company/southbridge/blog/548738/',
 'https://habr.com/ru/post/548732/',
 'https://habr.com/ru/post/548730/',
 'https://habr.com/ru/company/pixonic/blog/548728/',
 'https://habr.com/ru/company/droider/blog/538750/',
 'https://habr.com/ru/company/itelma/blog/548724/',
 'https://habr.com/ru/company/Voximplant/blog/548552/',
 'https://habr.com/ru/post/548720/',
 'https://habr.com/ru/post/548716/',
 'https://habr.com/ru/company/timeweb/blog/548712/',
 'https://habr.com/ru/company/selectel/blog/547944/',
 'https://habr.com/ru/company/habr/blog/548706/',
 'https://habr.com/ru/post/548700/',
 'https://habr.com/ru/company/cft/blog/547650/',
 'https://habr.com/ru/company/ruvds/blog/548692/',
 'https://habr.com/ru/company/vdsina/blog/548556/',
 'https://habr.com/ru/company/infowatch/blog/548688/',
 'https://habr.com/ru/post/548684/']

In [26]:
# достаем текст для каждого поста
posts_text = []
for link in posts_link:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    text = soup.find('div', class_='post__text').text
    posts_text.append(text)

In [37]:
# создаем датафрейм для всех постов с колонками date, title, link, text
# проверяя есть ли в заголовке или в тексте статьи ключевые слова KEYWORDS
habr_posts = pd.DataFrame()
for date, title, link, text in zip(posts_date, posts_title, posts_link, posts_text):
    if any(keyword.lower() in text.lower() for keyword in KEYWORDS) or any(keyword.lower() in title.lower() for keyword in KEYWORDS):
        row = {'date': date.strftime('%d %b %Y'), 'title': title, 'link': link, 'text': text}
        habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])
habr_posts.reset_index().drop(columns='index')

,date,title,link,text
0,24 Mar 2021,Linux для macOS M1: что сделала команда Asahi ...,https://habr.com/ru/post/548744/,Продолжаю перевод отчёта проекта Asahi Linux.П...
1,24 Mar 2021,Основы работы с Helm чартами и темплейтами — Ч...,https://habr.com/ru/post/548720/,"В этом руководстве мы кратко обсудим, как Helm..."


## Задание 2
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck.

In [54]:
EMAIL = ['xxxxx@x.ru', 'yyyyy@y.com']
# ссылка на скрытый API, к которому обращается avast при проверке email
hidden_api_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
# обязательные хэдеры
HEADERS = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}

In [55]:
# извлекаем инфу по утечкам в виде json
email_check = requests.post(hidden_api_url, json={'emailAddresses': EMAIL}, headers=HEADERS)
email_check.text

'{"breaches":{"88":{"breachId":88,"site":"ir.netease.com","recordsCount":256475863,"description":"In October 2015, the Chinese internet and gaming company NetEase suffered a data breach that leaked hundreds of millions of user credentials. The dump contained users\' email addresses along with their plain-text passwords.","publishDate":"2016-11-07T00:00:00Z","statistics":{"usernames":0,"passwords":255433278,"emails":256475863}},"16587":{"breachId":16587,"site":"dubsmash.com","recordsCount":161162574,"description":"In December 2018, Dubsmash\'s database was allegedly breached. The stolen data contains usernames, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.","publishDate":"2019-03-07T00:00:00Z","statistics":{"usernames":159256428,"passwords":161154927,"emails":161146252}},"17075":{"breachId":17075,"site":"toondo.com","recordsCount":6034161,"description":"In July 2019, the online comic creation site ToonDoo was alleg

In [69]:
# превращаем json в словарь
email_check_dict = json.loads(email_check.text)
email_check_dict.keys()

dict_keys(['breaches', 'data', 'summary'])

In [119]:
email_check_pd = pd.DataFrame()
# создаем датафрейм со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>
for breach_id, breach_source in zip(email_check_dict['breaches'], email_check_dict['data']):
    row = {
        'email': list(email_check_dict['data'][breach_source].keys())[0],
        'date': pd.to_datetime(email_check_dict['breaches'][breach_id]['publishDate'], format='%Y-%m-%d').strftime('%Y-%m-%d'),
        'breach_source': breach_source,
        'description': email_check_dict['breaches'][breach_id]['description']
    }
    email_check_pd = pd.concat([email_check_pd, pd.DataFrame([row])])
email_check_pd.reset_index().drop(columns='index')

,email,date,breach_source,description
0,yyyyy@y.com,2016-11-07,havenly.com,"In October 2015, the Chinese internet and gami..."
1,yyyyy@y.com,2019-03-07,adobe.com,"In December 2018, Dubsmash's database was alle..."
2,yyyyy@y.com,2019-11-14,ir.netease.com,"In July 2019, the online comic creation site T..."
3,yyyyy@y.com,2017-06-28,houdao.com,"In April 2011, gaming forum Houdao's user data..."
4,yyyyy@y.com,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
5,yyyyy@y.com,2020-08-06,dubsmash.com,"In June 2020, the online interior design servi..."
6,xxxxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
7,yyyyy@y.com,2016-10-21,toondo.com,"In October of 2013, criminals penetrated Adobe..."


## Дополнительная часть
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

In [2]:
GROUP = 'netology'
with open('vk_acces_token.txt', 'r') as token:
    TOKEN = token.read().strip()

In [3]:
GROUP_REQUEST = 'https://api.vk.com/method/wall.get'
VERSION = '5.58'
SLEEP = 0.33

In [25]:
params = {
    'owner_id': -30159897,
    'domain': f'https://vk.com/{GROUP}',
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50
}

In [26]:
vk_posts = requests.get(GROUP_REQUEST, params)
vk_posts

<Response [200]>

In [28]:
vk_netology_posts = pd.DataFrame()
# создаем датафрейм со столбцами: <дата> - <текст_поста>
for post in vk_posts.json()['response']['items']:
    row = {
        'date': pd.to_datetime(post['date'], unit='s'),
        'text': post['text']
    }
    vk_netology_posts = pd.concat([vk_netology_posts, pd.DataFrame([row])])
vk_netology_posts.reset_index().drop(columns='index')

,date,text
0,2021-03-17 13:25:03,Весна — лучшее время для перемен. Чтобы узнать...
1,2021-03-28 09:23:00,"Всё, что окружает нас в повседневной жизни так..."
2,2021-03-27 12:57:00,В этой афише мероприятий онлайн-эфир о развити...
3,2021-03-27 09:08:00,Разместить ссылку на свой сайт — это тоже свое...
4,2021-03-26 13:57:00,"Каждый день дизайнеры меняют IT и digital, фор..."
5,2021-03-26 07:44:00,"Точно решили, кем хотите стать в диджитале, но..."
6,2021-03-25 15:17:00,Мудборд — неотъемлемая часть подготовки любого...
7,2021-03-25 08:10:00,"Почему Java лучший выбор для новичка, какие на..."
8,2021-03-24 13:57:00,Что для вас важнее всего в работе? \nКому-то д...
9,2021-03-24 07:27:00,🚀 Запустили бесплатный курс «Основы Figma» \n ...
